# Import and concatenate

In [11]:
import pandas as pd 
import numpy as np
import os 
import requests
import json
from datetime import datetime
import glob

In [30]:
# This grabs all CSV files inside the "2022" folder
files = glob.glob("2022/*.csv")

In [32]:
# Load each CSV and combine into one DataFrame
df_list = [pd.read_csv(file) for file in files]
citibike_df = pd.concat(df_list, ignore_index=True)


In [34]:
# Extract date from 'started_at'
citibike_df['started_at'] = pd.to_datetime(citibike_df['started_at'])
citibike_df['date'] = citibike_df['started_at'].dt.date

In [36]:
# ✅ Preview the result
citibike_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date
0,7830EBE775F2EE25,classic_bike,2021-06-21 07:38:42,2021-06-21 10:49:25,Dixon Mills,JC076,NaN,NaN,40.721630,-74.049968,NaN,NaN,member,2021-06-21
1,629A3BFD50BF8392,docked_bike,2021-06-12 10:25:01,2021-06-12 10:35:11,Dixon Mills,JC076,Journal Square,JC103,40.721630,-74.049967,40.733670,-74.062500,member,2021-06-12
2,CF02EC434989A99D,docked_bike,2021-06-07 14:52:05,2021-06-07 14:58:25,11 St & Washington St,HB502,Hoboken Terminal - Hudson St & Hudson Pl,HB101,40.749984,-74.027150,40.735938,-74.030305,member,2021-06-07
3,E1BB8FED29AA5BA1,docked_bike,2021-06-07 08:29:19,2021-06-07 08:34:17,8 St & Washington St,HB603,Hoboken Terminal - Hudson St & Hudson Pl,HB101,40.745983,-74.028199,40.735938,-74.030305,casual,2021-06-07
4,787C59EE76D96BCA,docked_bike,2021-06-18 14:16:27,2021-06-18 14:26:26,11 St & Washington St,HB502,Columbus Park - Clinton St & 9 St,HB501,40.749984,-74.027150,40.748161,-74.032453,member,2021-06-18


In [38]:
citibike_df.shape

(1388898, 14)

This code reads all 2022 CitiBike monthly trip files, concatenates them into one DataFrame, and adds a date column for grouping.

# Fetch Weather Data via NOAA API

In [58]:
token = 'IkCWCgYCQyYduQYjBjDiTZCQLBXzLNdN'
headers = {'token': token}

params = {
    'datasetid': 'GHCND',
    'stationid': 'GHCND:USW00014732',  # LaGuardia Airport
    'startdate': '2022-01-01',
    'enddate': '2022-12-31',
    'limit': 1000,
    'units': 'standard'
}

response = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data", headers=headers, params=params)
weather_data = response.json()


In [59]:
# export to csv
with open('weather_2022.json', 'w') as f:
    json.dump(weather_data, f)


In [80]:
weather_df = pd.read_csv("weather_2022.json")
weather_df['date'] = pd.to_datetime(weather_df['date']).dt.date


KeyError: 'date'

In [64]:
# Load raw JSON data from NOAA
with open("weather_2022.json", "r") as f:
    data = json.load(f)


In [66]:
# Extract the 'results' list
weather_records = data.get('results', [])


In [68]:
# Convert to DataFrame
weather_raw_df = pd.DataFrame(weather_records)

In [70]:
# Convert 'date' to datetime and keep only the date part
weather_raw_df['date'] = pd.to_datetime(weather_raw_df['date']).dt.date

In [90]:
# Pivot so each weather type (TMAX, TMIN, PRCP, etc.) is a column
weather_df = weather_raw_df.pivot_table(
    index='date',
    columns='datatype',
    values='value',
    aggfunc='first'  # If there's a duplicate, take the first one
).reset_index()

In [92]:
# Display the final DataFrame
weather_df.head()

datatype,date,ADPT,ASLP,ASTP,AWBT,AWND,PRCP,RHAV,RHMN,RHMX,...,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT04,WT06,WT08,WT09
0,2022-01-01,100.0,10078.0,10078.0,106.0,6.3,0.76,89.0,83.0,93.0,...,200.0,200.0,14.1,17.0,1.0,NaN,NaN,NaN,NaN,NaN
1,2022-01-02,67.0,10051.0,10024.0,83.0,9.6,0.04,80.0,63.0,93.0,...,310.0,310.0,28.0,33.1,1.0,NaN,NaN,NaN,NaN,NaN
2,2022-01-03,-100.0,10196.0,10169.0,-39.0,14.3,0.00,49.0,38.0,62.0,...,340.0,350.0,28.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-04,-117.0,10281.0,10268.0,-44.0,8.7,0.00,48.0,35.0,60.0,...,320.0,350.0,19.9,27.1,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-05,6.0,10132.0,10139.0,33.0,7.6,0.24,76.0,52.0,86.0,...,250.0,200.0,15.0,23.9,1.0,NaN,NaN,NaN,1.0,NaN


# Merge and Export Final Data

In [76]:
# Assume weather_df has 'date', 'temp', 'precip'
merged_df = citibike_df.merge(weather_df, on='date', how='left')
merged_df.to_csv('citibike_weather_2022.csv', index=False)
